<a href="https://colab.research.google.com/github/test4373/My-Colab/blob/main/Music_Source_Separation_Training_(Turkish).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ses ayırma sistemi

In [ ]:
import base64
#@title kurulum (önce bunu çalıştır ve ayrılan dosyaları drive a kaydetmek için drive izni isticek hepsine tamam diyin)
# Sessiz kurulum için gerekli komutlar

from google.colab import drive
drive.mount('/content/drive')

%cd /content
!git clone https://github.com/ZFTurbo/Music-Source-Separation-Training

# Base64 encoded requirements
req = 'IyB0b3JjaCAjPT0yLjAuMQ0KbnVtcHkNCnBhbmRhcw0Kc2NpcHkNCnNvdW5kZmlsZQ0KbWxfY29sbGVjdGlvbnMNCnRxZG0NCnNlZ21lbnRhdGlvbl9tb2RlbHNfcHl0b3JjaD09MC4zLjMNCnRpbT0wLjkuMg0KYXVkaW9tZW50YXRpb25zPT0wLjI0LjANCnBlZGFsYm9hcmQ9PTAuOC4xDQpvbWVnYWNvbmY9PTIuMi4zDQpiZWFydHlwZT09MC4xNC4xDQpyb3RhcnlfZW1iZWRkaW5nX3RvcmNoPT0wLjMuNQ0KZWlub3BzPT0wLjYuMQ0KbGlicm9zYQ0KZGVtdWNzDQp0b3JjaG1ldHJpY3M9PTAuMTEuNA0Kc3BhZmU9PTAuMy4yDQpwcm90b2J1Zj09My4yMC4zDQp0b3JjaF9hdWRpb21lbnRhdGlvbnMNCmFzdGVyb2lkPT0wLjcuMA0KYXVyYWxvcw0KdG9yY2hzZWcNCg=='

# Requirements dosyasını oluştur
dec_req = base64.b64decode(req).decode('utf-8')
with open("Music-Source-Separation-Training/requirements.txt", "w") as f:
    f.write(dec_req)

# Klasör oluştur
!mkdir -p '/content/Music-Source-Separation-Training/ckpts'

%cd /content/Music-Source-Separation-Training

import subprocess
import concurrent.futures
import os
import sys

def install_package(package):
    """Hızlandırılmış paket kurulum fonksiyonu"""
    try:
        # Daha hızlı ve sessiz kurulum
        cmd = f'pip install -q --disable-pip-version-check --no-cache-dir {package}'
        result = subprocess.run(
            cmd,
            shell=True,
            capture_output=True,
            text=True,
            timeout=180  # 3 dakika timeout
        )

        return f'✅ {package}' if result.returncode == 0 else f'❌ {package}'
    except subprocess.TimeoutExpired:
        return f'⏰ {package} (Zaman aşımı)'
    except Exception as e:
        return f'❌ {package} (Hata: {str(e)})'

def install_packages():
    # Optimize edilmiş paket listesi
    packages = [
        # Temel kütüphaneler
        'pip', 'setuptools', 'wheel',

        # Veri işleme
        'numpy', 'pandas', 'scipy',

        # Makine öğrenmesi
        'torch', 'torchvision', 'ml_collections',
        'segmentation_models_pytorch',
        'torchmetrics', 'einops',

        # Görsel ve ses işleme
        'matplotlib', 'librosa',
        'soundfile', 'demucs',
        'audiomentations', 'torch_audiomentations',

        # Web ve API
        'httpx', 'openai', 'gradio',
        'google-api-python-client',
        'validators', 'pytube',

        # Yardımcı araçlar
        'tqdm', 'psutil',
        'omegaconf', 'beartype',

        # Ekstra kütüphaneler
        'pedalboard', 'spafe',
        'asteroid', 'auralos', 'torchseg', 'python-magic' , 'yt_dlp', 'yt_dlp[ffmpeg]',
        'yt_dlp' , 'rotary_embedding_torch'
    ]

    # Hızlandırma ortam değişkenleri
    os.environ.update({
        'PIP_DISABLE_PIP_VERSION_CHECK': '1',
        'PIP_NO_CACHE_DIR': '1',
        'PYTHONWARNINGS': 'ignore'
    })

    # Önce pip ve temel araçları güncelle
    subprocess.run('pip install -U pip setuptools wheel', shell=True, check=True)

    # Mevcut CPU çekirdek sayısını al
    import multiprocessing
    max_workers = min(multiprocessing.cpu_count(), 8)

    print(f"🚀 {max_workers} iş parçacığı ile hızlı kurulum başlatılıyor...")

    # Eş zamanlı kurulum
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Paketleri eş zamanlı kur
        results = list(executor.map(install_package, packages))

    # Sonuçları raporla
    print("\n📦 Kurulum Raporu:")
    for result in results:
        print(result)

    # Sistem güncellemeleri
    subprocess.run('apt-get update -qq && apt-get install -qq ffmpeg', shell=True)

    print("\n✨ Tüm paketler kuruldu!")

# Güvenli çalıştırma
if __name__ == '__main__':
    try:
        install_packages()
    except Exception as e:
        print(f"❌ Kritik kurulum hatası: {e}")
        sys.exit(1)

#şimdi de bunu çalıştır

In [ ]:

%cd /content/Music-Source-Separation-Training
#@markdown ## (sana bir link vericek onu aç örnek olarak: https://RastgeleSayılar.gradio.live)
import os
import torch
import yaml
import gradio as gr
import subprocess
import threading
import random
import time
import shutil
import librosa
import soundfile as sf
import numpy as np
import requests
import json
import locale
import shutil
from datetime import datetime
import glob
import subprocess
import yt_dlp
import validators
from pytube import YouTube
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import io
import math
import hashlib
import requests
import re
import gc
import psutil
import concurrent.futures
from tqdm import tqdm
from google.oauth2.credentials import Credentials
import tempfile
import requests
import os
from urllib.parse import urlparse

!mkdir /content/Music-Source-Separation-Training/input
!mkdir /content/Music-Source-Separation-Training/output


def update_progress(progress=gr.Progress()):
    def track_progress(percent):
        progress(percent/100)
    return track_progress


def clear_input_folder():
    # Klasörü temizleme işlemi
    input_path = "/content/Music-Source-Separation-Training/input"
    if os.path.exists(input_path):
        shutil.rmtree(input_path)
    os.makedirs(input_path, exist_ok=True)


import os
import shutil
import time
import validators
import yt_dlp
import gdown

def download_callback(url, download_type='direct'):
    try:
        # Klasörü temizle
        clear_input_folder()

        # Hedef klasör
        input_path = "/content/Music-Source-Separation-Training/input"
        os.makedirs(input_path, exist_ok=True)

        # URL kontrolü
        if not validators.url(url):
            return None, "Geçersiz URL", None, None

        # İndirme türüne göre farklı işlemler
        if download_type == 'direct':
            ydl_opts = {
                'format': 'bestaudio/best',
                'outtmpl': os.path.join(input_path, '%(title)s.%(ext)s'),
                'postprocessors': [{
                    'key': 'FFmpegExtractAudio',
                    'preferredcodec': 'wav',
                    'preferredquality': '0',
                }],
                'max_filesize': 10 * 1024 * 1024 * 1024,  # 10 GB sınırı
                'nooverwrites': True,
                'no_color': True,
                'progress_hooks': [download_progress_hook]
            }

            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                info_dict = ydl.extract_info(url, download=True)
                wav_path = ydl.prepare_filename(info_dict).replace(f".{info_dict['ext']}", ".wav")

        elif download_type == 'drive':
            # Google Drive'dan büyük dosya indirmek için gdown kullan
            file_id = url.split("/")[-2] if "/file/d/" in url else url.split("=")[-1]
            output_path = os.path.join(input_path, "downloaded_file.wav")
            gdown.download(f'https://drive.google.com/uc?id={file_id}', output_path, quiet=False)

            # İndirme sonrası dosya kontrolü
            if os.path.exists(output_path):
                wav_path = output_path
            else:
                return None, "Dosya indirilemedi", None, None

        else:
            return None, "Geçersiz indirme türü", None, None

        # Dosya kontrolleri
        if wav_path and os.path.exists(wav_path):
            filename = os.path.basename(wav_path)
            input_file_path = os.path.join(input_path, filename)

            # Aynı isimde dosya varsa timestamp ekle
            if os.path.exists(input_file_path):
                base, ext = os.path.splitext(filename)
                timestamp = int(time.time())
                filename = f"{base}_{timestamp}{ext}"
                input_file_path = os.path.join(input_path, filename)

            # Dosyayı taşı
            shutil.move(wav_path, input_file_path)

            return (
                gr.File(value=input_file_path),  # İndirilen dosya
                f"Başarıyla indirildi: {filename}",  # Mesaj
                gr.File(value=input_file_path),  # input_audio güncellemesi
                gr.Audio(value=input_file_path)  # original_audio için ses
            )

        return None, "Dosya indirilemedi", None, None

    except Exception as e:
        print(f"Download hatası: {e}")
        return None, str(e), None, None

# İndirme ilerlemesini takip etmek için hook fonksiyonu
def download_progress_hook(d):
    if d['status'] == 'finished':
        print('İndirme tamamlandı, dönüştürme yapılıyor...')
    elif d['status'] == 'downloading':
        downloaded_bytes = d.get('downloaded_bytes', 0)
        total_bytes = d.get('total_bytes') or d.get('total_bytes_estimate', 0)
        if total_bytes > 0:
            percent = downloaded_bytes * 100. / total_bytes
            print(f'İndiriliyor: {percent:.1f}%')

# Global değişkeni en üstte tanımlayın
INPUT_DIR = "/content/Music-Source-Separation-Training/input"

def download_file(url, directory='ckpts'):
    """
    Belirtilen URL'den dosya indirir

    Args:
        url (str): İndirilecek dosyanın URL'si
        directory (str, optional): İndirilen dosyanın kaydedileceği dizin. Varsayılan 'ckpts'

    Returns:
        str: İndirilen dosyanın tam yolu
    """
    # Dizini oluştur
    os.makedirs(directory, exist_ok=True)

    # Dosya adını URL'den çıkar
    filename = os.path.basename(url)
    filepath = os.path.join(directory, filename)

    # Eğer dosya zaten varsa indirme
    if os.path.exists(filepath):
        print(f"{filename} zaten mevcut.")
        return filepath

    try:
        # İndirme işlemi
        response = requests.get(url, stream=True)
        response.raise_for_status()  # HTTP hata kontrolü

        # Dosyayı kaydet
        with open(filepath, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

        print(f"{filename} başarıyla indirildi.")
        return filepath

    except Exception as e:
        print(f"{filename} indirilemedi: {e}")
        return None

        clear_memory()


def generate_random_port():
    return random.randint(1000, 9000)

    clear_memory()

# Markdown açıklamaları
markdown_intro = """
# Ses Ayrıştırma Aracı

Bu araç, ses dosyalarını ayrıştırmak için kullanılır.
"""

class IndentDumper(yaml.Dumper):
    def increase_indent(self, flow=False, indentless=False):
        return super(IndentDumper, self).increase_indent(flow, False)

def tuple_constructor(loader, node):
    values = loader.construct_sequence(node)
    return tuple(values)

yaml.SafeLoader.add_constructor('tag:yaml.org,2002:python/tuple', tuple_constructor)

def conf_edit(config_path, chunk_size, overlap):
    print("Using custom overlap/chunk_size values")
    with open(config_path, 'r') as f:
        data = yaml.load(f, Loader=yaml.SafeLoader)

    if 'use_amp' not in data.keys():
        data['training']['use_amp'] = True

    data['audio']['chunk_size'] = chunk_size
    data['inference']['num_overlap'] = overlap

    with open(config_path, 'w') as f:
        yaml.dump(data, f, default_flow_style=False, sort_keys=False, Dumper=IndentDumper, allow_unicode=True)


def save_uploaded_file(uploaded_file, is_input=False):
    """Yüklenen dosyayı belirtilen dizine kaydeder."""
    try:
        # Dosya adını güvenli bir şekilde al
        if hasattr(uploaded_file, 'name'):
            file_name = os.path.basename(uploaded_file.name)
        else:
            file_name = os.path.basename(str(uploaded_file))

        # Girdi dizinini belirle
        target_directory = INPUT_DIR if is_input else OUTPUT_DIR

        # Hedef dosya yolunu oluştur
        target_path = os.path.join(target_directory, file_name)

        # Dosyayı kaydet
        if hasattr(uploaded_file, 'read'):
            # Gradio dosya nesnesi
            with open(target_path, "wb") as f:
                f.write(uploaded_file.read())
        else:
            # Zaten bir dosya yolu ise
            shutil.copy(uploaded_file, target_path)

        print(f"{file_name} başarıyla kaydedildi: {target_path}")
        return target_path
    except Exception as e:
        print(f"Dosya kaydedilirken hata oluştu: {e}")
        return None

        clear_memory()

def extract_model_name(full_model_string):
    """
    Model adını temizleyen fonksiyon
    """
    if not full_model_string:
        return ""

    cleaned = str(full_model_string)

    # Açıklama kısmını çıkar
    if ' - ' in cleaned:
        cleaned = cleaned.split(' - ')[0]

    # Emoji prefixlerini çıkar
    emoji_prefixes = ['✅ ', '👥 ', '🏛️ ', '🔇 ', '🔉 ', '🎬 ', '🎼 ']
    for prefix in emoji_prefixes:
        if cleaned.startswith(prefix):
            cleaned = cleaned[len(prefix):]

    return cleaned.strip()

BASE_PATH = '/content/Music-Source-Separation-Training'
INPUT_DIR = os.path.join(BASE_PATH, 'input')
OUTPUT_DIR = '/content/drive/MyDrive/output'

def clear_directory(directory):
    """Verilen dizindeki tüm dosyaları siler."""
    files = glob.glob(os.path.join(directory, '*'))  # Dizin içindeki tüm dosyaları al
    for f in files:
        try:
            os.remove(f)  # Dosyayı sil
        except Exception as e:
            print(f"{f} silinemedi: {e}")

            clear_memory()

def create_directory(directory):
    """Verilen dizini oluşturur (varsa, yoksa)."""
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"{directory} dizini oluşturuldu.")

        clear_memory()

def process_audio(input_audio, model, chunk_size, overlap, flac_file, use_tta, pcm_type):
    # Girdi ve çıktı dizinlerini oluştur
    create_directory(INPUT_DIR)
    create_directory(OUTPUT_DIR)

    # Mevcut dosyaları sil
    clear_directory(INPUT_DIR)
    clear_directory(OUTPUT_DIR)

    # Model ismini temizle
    clean_model = extract_model_name(model)
    print(f"Temizlenmiş Model İsmi: {clean_model}")

    # Dosya kontrolü
    if input_audio is None:
        print("Dosya yüklenmedi")
        return None, None, None, None, None, None, None, None, None

    # Dosyayı kaydet
    dest_path = save_uploaded_file(input_audio, is_input=True)

    if not dest_path:
        print("Dosya kaydedilemedi")
        return None, None, None, None, None, None, None, None, None

    # input_folder ve output_folder tanımla
    input_folder = INPUT_DIR
    output_folder = OUTPUT_DIR

    # Model seçimi ve ilgili parametreleri belirle
    model_type, config_path, start_check_point = "", "", ""

    if clean_model == 'VOCALS-InstVocHQ':
            model_type = 'mdx23c'
            config_path = 'ckpts/config_vocals_mdx23c.yaml'
            start_check_point = 'ckpts/model_vocals_mdx23c_sdr_10.17.ckpt'
            download_file('https://raw.githubusercontent.com/ZFTurbo/Music-Source-Separation-Training/main/configs/config_vocals_mdx23c.yaml')
            download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v1.0.0/model_vocals_mdx23c_sdr_10.17.ckpt')

    elif clean_model == 'VOCALS-MelBand-Roformer (by KimberleyJSN)':
            model_type = 'mel_band_roformer'
            config_path = 'ckpts/config_vocals_mel_band_roformer_kj.yaml'
            start_check_point = 'ckpts/MelBandRoformer.ckpt'
            download_file('https://raw.githubusercontent.com/ZFTurbo/Music-Source-Separation-Training/main/configs/KimberleyJensen/config_vocals_mel_band_roformer_kj.yaml')
            download_file('https://huggingface.co/KimberleyJSN/melbandroformer/resolve/main/MelBandRoformer.ckpt')
            conf_edit(config_path, chunk_size, overlap)

    elif clean_model == 'VOCALS-BS-Roformer_1297 (by viperx)':
            model_type = 'bs_roformer'
            config_path = 'ckpts/model_bs_roformer_ep_317_sdr_12.9755.yaml'
            start_check_point = 'ckpts/model_bs_roformer_ep_317_sdr_12.9755.ckpt'
            download_file('https://raw.githubusercontent.com/ZFTurbo/Music-Source-Separation-Training/main/configs/viperx/model_bs_roformer_ep_317_sdr_12.9755.yaml')
            download_file('https://github.com/TRvlvr/model_repo/releases/download/all_public_uvr_models/model_bs_roformer_ep_317_sdr_12.9755.ckpt')
            conf_edit(config_path, chunk_size, overlap)

    elif clean_model == 'VOCALS-BS-Roformer_1296 (by viperx)':
            model_type = 'bs_roformer'
            config_path = 'ckpts/model_bs_roformer_ep_368_sdr_12.9628.yaml'
            start_check_point = 'ckpts/model_bs_roformer_ep_368_sdr_12.9628.ckpt'
            download_file('https://github.com/TRvlvr/model_repo/releases/download/all_public_uvr_models/model_bs_roformer_ep_368_sdr_12.9628.ckpt')
            download_file('https://raw.githubusercontent.com/TRvlvr/application_data/main/mdx_model_data/mdx_c_configs/model_bs_roformer_ep_368_sdr_12.9628.yaml')
            conf_edit(config_path, chunk_size, overlap)

    elif clean_model == 'VOCALS-BS-RoformerLargev1 (by unwa)':
            model_type = 'bs_roformer'
            config_path = 'ckpts/config_bsrofoL.yaml'
            start_check_point = 'ckpts/BS-Roformer_LargeV1.ckpt'
            download_file('https://huggingface.co/jarredou/unwa_bs_roformer/resolve/main/BS-Roformer_LargeV1.ckpt')
            download_file('https://huggingface.co/jarredou/unwa_bs_roformer/raw/main/config_bsrofoL.yaml')
            conf_edit(config_path, chunk_size, overlap)

    elif clean_model == 'VOCALS-Mel-Roformer big beta 4 (by unwa)':
            model_type = 'mel_band_roformer'
            config_path = 'ckpts/config_melbandroformer_big_beta4.yaml'
            start_check_point = 'ckpts/melband_roformer_big_beta4.ckpt'
            download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-big/resolve/main/melband_roformer_big_beta4.ckpt')
            download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-big/raw/main/config_melbandroformer_big_beta4.yaml')
            conf_edit(config_path, chunk_size, overlap)

    elif clean_model == 'INST-Mel-Roformer v1 (by unwa)':
            model_type = 'mel_band_roformer'
            config_path = 'ckpts/config_melbandroformer_inst.yaml'
            start_check_point = 'ckpts/m elband_roformer_inst_v1.ckpt'
            download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-Inst/resolve/main/melband_roformer_inst_v1.ckpt')
            download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-Inst/raw/main/config_melbandroformer_inst.yaml')
            conf_edit(config_path, chunk_size, overlap)

    elif clean_model == 'INST-Mel-Roformer v2 (by unwa)':
            model_type = 'mel_band_roformer'
            config_path = 'ckpts/config_melbandroformer_inst_v2.yaml'
            start_check_point = 'ckpts/melband_roformer_inst_v2.ckpt'
            download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-Inst/resolve/main/melband_roformer_inst_v2.ckpt')
            download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-Inst/raw/main/config_melbandroformer_inst_v2.yaml')
            conf_edit(config_path, chunk_size, overlap)

    elif clean_model == 'INST-VOC-Mel-Roformer a.k.a. duality (by unwa)':
            model_type = 'mel_band_roformer'
            config_path = 'ckpts/config_melbandroformer_instvoc_duality.yaml'
            start_check_point = 'ckpts/melband_roformer_instvoc_duality_v1.ckpt'
            download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-InstVoc-Duality/resolve/main/melband_roformer_instvoc_duality_v1.ckpt')
            download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-InstVoc-Duality/raw/main/config_melbandroformer_instvoc_duality.yaml')
            conf_edit(config_path, chunk_size, overlap)

    elif clean_model == 'INST-VOC-Mel-Roformer a.k.a. duality v2 (by unwa)':
            model_type = 'mel_band_roformer'
            config_path = 'ckpts/config_melbandroformer_instvoc_duality.yaml'
            start_check_point = 'ckpts/melband_roformer_instvox_duality_v2.ckpt'
            download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-InstVoc-Duality/resolve/main/melband_roformer_instvox_duality_v2.ckpt')
            download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-InstVoc-Duality/raw/main/config_melbandroformer_instvoc_duality.yaml')
            conf_edit(config_path, chunk_size, overlap)

    elif clean_model == 'KARAOKE-MelBand-Roformer (by aufr33 & viperx)':
            model_type = 'mel_band_roformer'
            config_path = 'ckpts/config_mel_band_roformer_karaoke.yaml'
            start_check_point = 'ckpts/mel_band_roformer_karaoke_aufr33_viperx_sdr_10.1956.ckpt'
            download_file('https://huggingface.co/jarredou/aufr33-viperx-karaoke-melroformer-model/resolve/main/mel_band_roformer_karaoke_aufr33_viperx_sdr_10.1956.ckpt')
            download_file('https://huggingface.co/jarredou/aufr33-viperx-karaoke-melroformer-model/resolve/main/config_mel_band_roformer_karaoke.yaml')
            conf_edit(config_path, chunk_size, overlap)

    elif clean_model == 'OTHER-BS-Roformer_1053 (by viperx)':
            model_type = 'bs_roformer'
            config_path = 'ckpts/model_bs_roformer_ep_937_sdr_10.5309.yaml'
            start_check_point = 'ckpts/model_bs_roformer_ep_937_sdr_10.5309.ckpt'
            download_file('https://github.com/TRvlvr/model_repo/releases/download/all_public_uvr_models/model_bs_roformer_ep_937_sdr_10.5309.ckpt')
            download_file('https://raw.githubusercontent.com/TRvlvr/application_data/main/mdx_model_data/mdx_c_configs/model_bs_roformer_ep_937_sdr_10.5309.yaml')
            conf_edit(config_path, chunk_size, overlap)

    elif clean_model == 'CROWD-REMOVAL-MelBand-Roformer (by aufr33)':
            model_type = 'mel_band_roformer'
            config_path = 'ckpts/model_mel_band_roformer _crowd.yaml'
            start_check_point = 'ckpts/mel_band_roformer_crowd_aufr33_viperx_sdr_8.7144.ckpt'
            download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v.1.0.4/mel_band_roformer_crowd_aufr33_viperx_sdr_8.7144.ckpt')
            download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v.1.0.4/model_mel_band_roformer_crowd.yaml')
            conf_edit(config_path, chunk_size, overlap)

    elif clean_model == 'VOCALS-VitLarge23 (by ZFTurbo)':
            model_type = 'segm_models'
            config_path = 'ckpts/config_vocals_segm_models.yaml'
            start_check_point = 'ckpts/model_vocals_segm_models_sdr_9.77.ckpt'
            download_file('https://raw.githubusercontent.com/ZFTurbo/Music-Source-Separation-Training/refs/heads/main/configs/config_vocals_segm_models.yaml')
            download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v1.0.0/model_vocals_segm_models_sdr_9.77.ckpt')

    elif clean_model == 'CINEMATIC-BandIt_Plus (by kwatcharasupat)':
            model_type = 'bandit'
            config_path = 'ckpts/config_dnr_bandit_bsrnn_multi_mus64.yaml'
            start_check_point = 'ckpts/model_bandit_plus_dnr_sdr_11.47.chpt'
            download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v.1.0.3/config_dnr_bandit_bsrnn_multi_mus64.yaml')
            download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v.1.0.3/model_bandit_plus_dnr_sdr_11.47.chpt')

    elif clean_model == 'DRUMSEP-MDX23C_DrumSep_6stem (by aufr33 & jarredou)':
            model_type = 'mdx23c'
            config_path = 'ckpts/aufr33-jarredou_DrumSep_model_mdx23c_ep_141_sdr_10.8059.yaml'
            start_check_point = 'ckpts/aufr33-jarredou_DrumSep_model_mdx23c_ep_141_sdr_10.8059.ckpt'
            download_file('https://github.com/jarredou/models/releases/download/aufr33-jarredou_MDX23C_DrumSep_model_v0.1/aufr33-jarredou_DrumSep_model_mdx23c_ep_141_sdr_10.8059.ckpt')
            download_file('https://github.com/jarredou/models/releases/download/aufr33-jarredou_MDX23C_DrumSep_model_v0.1/aufr33-jarredou_DrumSep_model_mdx23c_ep_141_sdr_10.8059.yaml')

    elif clean_model == '4STEMS-SCNet_MUSDB18 (by starrytong)':
            model_type = 'scnet'
            config_path = 'ckpts/config_musdb18_scnet.yaml'
            start_check_point = 'ckpts/scnet_checkpoint_musdb18.ckpt'
            download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v.1.0.6/config_musdb18_scnet.yaml')
            download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v.1.0.6/scnet_checkpoint_musdb18.ckpt')

    elif clean_model == 'DE-REVERB-MDX23C (by aufr33 & jarredou)':
            model_type = 'mdx23c'
            config_path = 'ckpts/config_dereverb_mdx23c.yaml'
            start_check_point = 'ckpts/dereverb_mdx23c_sdr_6.9096.ckpt'
            download_file('https://huggingface.co/jarredou/aufr33_jarredou_MDXv3_DeReverb/resolve/main/dereverb_mdx23c_sdr_6.9096.ckpt')
            download_file('https://huggingface.co/jarredou/aufr33_jarredou_MDXv3_DeReverb/resolve/main/config_dereverb_mdx23c.yaml')

    elif clean_model == 'DENOISE-MelBand-Roformer-1 (by aufr33)':
            model_type = 'mel_band_roformer'
            config_path = 'ckpts/model_mel_band_roformer_denoise.yaml'
            start_check_point = 'ckpts/denoise_mel_band_roformer_aufr33_sdr_27.9959.ckpt'
            download_file('https://huggingface.co/jarredou/aufr33_MelBand_Denoise/resolve/main/denoise_mel_band_roformer_aufr33_sdr_27.9959.ckpt')
            download_file('https://huggingface.co/jarredou/aufr33_MelBand_Denoise/resolve/main/model_mel_band_roformer_denoise.yaml')
            conf_edit(config_path, chunk_size, overlap)

    elif clean_model == 'DENOISE-MelBand-Roformer-2 (by aufr33)':
            model_type = 'mel_band_roformer'
            config_path = 'ckpts/model_mel_band_roformer_denoise.yaml'
            start_check_point = 'ckpts/denoise_mel_band_roformer_aufr33_aggr_sdr_27.9768.ckpt'
            download_file('https://huggingface.co/jarredou/aufr33_MelBand_Denoise/resolve/main/denoise_mel_band_roformer_aufr33_aggr_sdr_27.9768.ckpt')
            download_file('https://huggingface.co/jarredou/aufr33_MelBand_Denoise/resolve/main/model_mel_band_roformer_denoise.yaml')
            conf_edit(config_path, chunk_size, overlap)


    # Diğer model seçenekleri buraya eklenecek...
    # (Daha önceki kodda verdiğiniz tüm elif blokları buraya gelecek)

    else:
        print(f"Desteklenmeyen model: {clean_model}")
        return None, None, None, None, None, None, None, None, None


    # Komut parametrelerini hazırla
    cmd_parts = [
        "python", "inference.py",
        "--model_type", model_type,
        "--config_path", config_path,
        "--start_check_point", start_check_point,
        "--input_folder", input_folder,
        "--store_dir", output_folder,
        "--extract_instrumental"  # Her zaman instrumental çıkart
    ]


    # İsteğe bağlı parametreleri ekle
    optional_params = {
        'flac_file': flac_file,
        'use_tta': use_tta
    }


    for param, value in optional_params.items():
        if value:
            cmd_parts.append(f"--{param}")

    if pcm_type and pcm_type.strip():
        cmd_parts.extend(["--pcm_type", pcm_type.split()[1]])

    # Komutu çalıştır
    try:
        process = subprocess.Popen(
            cmd_parts,
            cwd=BASE_PATH,  # BASE_PATH değişkenini kullandım
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            bufsize=1,
            universal_newlines=True
        )


        # Çıktıları oku
        for line in process.stdout:
            print(line.strip())

        for line in process.stderr:
            print(line.strip())

        process.wait()


        # Çıktı dosyalarını bul
        output_files = os.listdir(output_folder)

        # Dört farklı stem dosyasını bul
        vocal_file = next((os.path.join(output_folder, f) for f in output_files if 'vocals' in f.lower()), None)
        drum_file = next((os.path.join(output_folder, f) for f in output_files if 'drum' in f.lower()), None)
        bass_file = next((os.path.join(output_folder, f) for f in output_files if 'bass' in f.lower()), None)
        other_file = next((os.path.join(output_folder, f) for f in output_files if 'other' in f.lower()), None)
        instrumental_file = next((os.path.join(output_folder, f) for f in output_files if 'instrumental' in f.lower()), None)
        effects_file = next((os.path.join(output_folder, f) for f in output_files if 'effects' in f.lower()), None)
        speech_file = next((os.path.join(output_folder, f) for f in output_files if 'speech' in f.lower()), None)
        music_file = next((os.path.join(output_folder, f) for f in output_files if 'music' in f.lower()), None)
        dry_file = next((os.path.join(output_folder, f) for f in output_files if 'dry' in f.lower()), None)

        # Her durumda beş değer döndür
        return vocal_file or None, instrumental_file or None, drum_file or None, bass_file or None, other_file or None, effects_file or None, speech_file or None, music_file or None, dry_file or None  # Tüm stem dosyalarını döndürün

    except Exception as e:
        print(f"Hata oluştu: {e}")
        return None, None, None, None, None, None, None, None, None  # Hata durumunda None döndürün


def create_interface():
    # Model seçeneklerini önceden tanımlayalım
    model_choices = {
        "Vocal Separation": [
            'VOCALS-BS-Roformer_1297 (by viperx)',
            '✅ VOCALS-Mel-Roformer big beta 4 (by unwa) - Melspectrogram tabanlı yüksek performans',
            '✅ VOCALS-BS-RoformerLargev1 (by unwa) - Geniş kapsamlı model',
            'VOCALS-InstVocHQ - Genel amaçlı model',
            'VOCALS-MelBand-Roformer (by KimberleyJSN) - Alternatif model',
            'VOCALS-VitLarge23 (by ZFTurbo) - Transformer tabanlı model'
        ],
        "Instrumental Separation": [
            '✅ INST-VOC-Mel-Roformer a.k.a. duality v2 (by unwa) - Son versiyon enstrüman ayrıştırma',
            '✅ INST-VOC-Mel-Roformer a.k.a. duality (by unwa) - Önceki versiyon',
            'INST-Separator MDX23C (by aufr33) - Alternatif enstrüman ayrıştırma',
            '✅ INST-Mel-Roformer v2 (by unwa) - En güncel enstrumental ayrıştırma modeli',
            '✅ INST-Mel-Roformer v1 (by unwa) - eski enstrumental ayrıştırma modeli'
        ],
        "Karaoke & Accompaniment": [
            '✅ KARAOKE-MelBand-Roformer (by aufr33 & viperx) - Gelişmiş karaoke ayrıştırma',
            'KARAOKE-BS-Roformer (by viperx) - Standart karaoke modeli'
        ],
        "Noise & Effect Removal": [
            '👥 CROWD-REMOVAL-MelBand-Roformer (by aufr33) - Kalabalık gürültüsü temizleme',
            '🏛️ DE-REVERB-MDX23C (by aufr33 & jarredou) - Yankı azaltma',
            '🔇 DENOISE-MelBand-Roformer-1 (by aufr33) - Temel gürültü azaltma',
            '🔉 DENOISE-MelBand-Roformer-2 (by aufr33) - Gelişmiş gürültü azaltma'
        ],
        "Drum Separation": [
            '✅ DRUMSEP-MDX23C_DrumSep_6stem (by aufr33 & jarredou) - Detaylı davul ayrıştırma'
        ],
        "Multi-Stem & Other Models": [
            '🎬 4STEMS-SCNet_MUSDB18 (by starrytong) - Çoklu stem ayırma',
            '🎼 CINEMATIC-BandIt_Plus (by kwatcharasupat) - Sinematik müzik analizi',
            'OTHER-BS-Roformer_1053 (by viperx) - Diğer özel modeller'
        ]
    }

    def update_models(category):
        models = model_choices.get(category, [])
        return gr.Dropdown(
            label="Model Seç",
            choices=models,
            value=models[0] if models else None
        )

    with gr.Blocks() as demo:
        gr.Markdown("# 🎵 Müzik Kaynak Ayırma Aracı")

        with gr.Tabs():
            with gr.Tab("Ses Ayırma"):
                with gr.Row():
                    with gr.Column(scale=1):
                        input_audio = gr.File(label="Ses Dosyası Seç", type="filepath")

                        model_category = gr.Dropdown(
                            label="Model Kategorisi",
                            choices=list(model_choices.keys())
                        )

                        model_dropdown = gr.Dropdown(label="Model Seç")

                        overlap = gr.Slider(
                        label="Overlap",
                        info="bu genelde 5 ila 2 arası olmalı farklı bişiler istersen değiştir.",
                        minimum=1,
                        maximum=10,
                        step=1,
                        value=3
                    )

                    model_category.change(
                        fn=update_models,
                        inputs=model_category,
                        outputs=model_dropdown
                    )

                    with gr.Column(scale=1):
                        chunk_size = gr.Dropdown(
                            label="Chunk Size",
                            info="buna dokunma.",
                            choices=[352800, 485100],
                            value=352800
                        )

                        flac_file = gr.Checkbox(
                            label="FLAC Dosyası",
                            info="bunu eğer flac çıktısı istersen aç"
                        )

                        use_tta = gr.Checkbox(
                            label="TTA Kullan",
                            info="Test Time Augmentation: Modelin tahmin performansını artırır. Aynı zamanda işlem süresini de uzatır."
                        )

                        pcm_type = gr.Dropdown(
                            label="PCM Tipi",
                            choices=[
                                'flac PCM_16',
                                'flac PCM_24',
                                'wav FLOAT'
                            ]
                        )

                        process_btn = gr.Button("Sesi İşle")

                        with gr.Column():
                            original_audio = gr.Audio(label="Orijinal Ses")
                            vocals_audio = gr.Audio(label="Vocals")
                            instrumental_audio = gr.Audio(label="Instrumental")
                            drum_audio = gr.Audio(label="Drum")
                            bass_audio = gr.Audio(label="Bass")
                            other_audio = gr.Audio(label="Other")
                            effects_audio = gr.Audio(label="effects")
                            speech_audio = gr.Audio(label="speech")
                            music_audio = gr.Audio(label="music")
                            dry_audio = gr.Audio(label="dry")

                input_audio.upload(
                    fn=lambda x: x,
                    inputs=input_audio,
                    outputs=original_audio
                )

                process_btn.click(
                    fn=process_audio,
                    inputs=[
                        input_audio, model_dropdown, chunk_size, overlap,
                        flac_file, use_tta, pcm_type
                    ],
                    outputs=[
                        vocals_audio,
                        instrumental_audio,
                        drum_audio,
                        bass_audio,
                        other_audio,
                        effects_audio,
                        speech_audio,
                        music_audio,
                        dry_audio
                    ]
                )
            with gr.Tab("İndirme"):
                gr.Markdown("## 🔗 Ses Dosyası İndirme")

                with gr.Row():
                    with gr.Column():
                        gr.Markdown("### 📂 Google Drive İndirme")
                        drive_url_input = gr.Textbox(label="Google Drive Linki")
                        drive_download_btn = gr.Button("İndir")
                        drive_download_status = gr.Textbox(label="Durum")
                        drive_download_output = gr.File(label="İndirilen Dosya")

                    with gr.Column():
                        gr.Markdown("### 🌐 Direkt URL İndirme")
                        direct_url_input = gr.Textbox(label="Direkt URL")
                        direct_download_btn = gr.Button("İndir")
                        direct_download_status = gr.Textbox(label="Durum")
                        direct_download_output = gr.File(label="İndirilen Dosya")

                drive_download_btn.click(
                    fn =download_callback,
                    inputs=[drive_url_input, gr.State('drive')],
                    outputs=[drive_download_output, drive_download_status, input_audio, original_audio]
                )

                direct_download_btn.click(
                    fn=download_callback,
                    inputs=[direct_url_input, gr.State('direct')],
                    outputs=[direct_download_output, direct_download_status, input_audio, original_audio]
                )
        # Diğer sekmeler buraya eklenebilir

    return demo

def launch_with_share():
    port = generate_random_port()
    demo = create_interface()
    share_link = demo.launch(
        share=True,
        server_port=port,
        server_name='0.0.0.0',
        inline=False,
        allowed_paths=['/content/drive/MyDrive/output']
    )
    try:
        while True:
            time.sleep(1)

    except KeyboardInterrupt:
        print("Döngü durduruldu.")

if __name__ == "__main__":
    launch_with_share()

#kalite artirma bölümü

In [ ]:
%%capture --no-stderr
#@title ###(önceki hücreyi durdur) şimdi napıcağını biliyosun önce gerekenleri indirmemiz gerek şimdi "↳ 1 hücre gizli" yazısının yanındaki yuvarlak olana bas
%cd /content/
!git clone https://github.com/JusperLee/Apollo.git && cd Apollo

!mkdir /content/Apollo/model
%cd /content/Apollo/model
!wget 'https://huggingface.co/JusperLee/Apollo/resolve/main/pytorch_model.bin'
!wget 'https://huggingface.co/jarredou/lew_apollo_vocal_enhancer/resolve/main/apollo_model.ckpt'
!wget 'https://huggingface.co/jarredou/lew_apollo_vocal_enhancer/resolve/main/apollo_model_v2.ckpt'

%cd /content/Apollo/configs
!wget 'https://huggingface.co/jarredou/lew_apollo_vocal_enhancer/resolve/main/config_apollo_vocal.yaml'

!rm -rf '/content/Apollo/inference.py'
%cd /content/Apollo
!wget 'https://raw.githubusercontent.com/jarredou/Apollo-Colab-Inference/main/inference.py'

!pip install omegaconf ml_collections

In [ ]:

#@title #Kalite artırma arayüzü içinde bunu çalıştır [ses ayırmayı tekrar kullanmak için bunu durdurmana gerek yok zaten kendisi duruyo)
import os
import shutil
from IPython.display import Audio, display
import ipywidgets as widgets
from IPython.display import display as ipydisplay

class AudioUpscaler:
    def __init__(self):
        # Model seçenekleri
        self.model_options = {
            'MP3 Enhancer': {
                'ckpt': '/content/Apollo/model/pytorch_model.bin',
                'config': 'configs/apollo.yaml'
            },
            'Lew Vocal Enhancer': {
                'ckpt': '/content/Apollo/model/apollo_model.ckpt',
                'config': 'configs/apollo.yaml'
            },
            'Lew Vocal Enhancer v2 (beta)': {
                'ckpt': '/content/Apollo/model/apollo_model_v2.ckpt',
                'config': 'configs/config_apollo_vocal.yaml'
            }
        }

        # Widget'ları oluştur
        self.create_widgets()

    def create_widgets(self):
        # Input dizini seçimi
        self.input_dir_widget = widgets.Text(
            value='/content/Music-Source-Separation-Training/output',
            description='Input Dizini:',
            layout=widgets.Layout(width='500px')
        )

        # Output dizini seçimi
        self.output_dir_widget = widgets.Text(
            value='/content/output',
            description='Output Dizini:',
            layout=widgets.Layout(width='500px')
        )

        # Model seçimi
        self.model_widget = widgets.Dropdown(
            options=list(self.model_options.keys()),
            value='Lew Vocal Enhancer v2 (beta)',
            description='Model:'
        )

        # Chunk size slider
        self.chunk_size_widget = widgets.IntSlider(
            value=25,
            min=3,
            max=25,
            step=1,
            description='Chunk Size:'
        )

        # Overlap slider
        self.overlap_widget = widgets.IntSlider(
            value=2,
            min=2,
            max=10,
            step=1,
            description='Overlap:'
        )

        # İşlem butonu
        self.process_button = widgets.Button(
            description='Sesleri Geliştir',
            button_style='primary'
        )
        self.process_button.on_click(self.process_files)

    def display_interface(self):
        ipydisplay(widgets.VBox([
            widgets.HTML("<h2>🎵 Ses Kalitesi Artırma Aracı 🎵</h2>"),
            self.input_dir_widget,
            self.output_dir_widget,
            self.model_widget,
            self.chunk_size_widget,
            self.overlap_widget,
            self.process_button
        ]))

    def process_files(self, b):
        # Output klasörünü oluştur
        input_dir = self.input_dir_widget.value
        output_dir = self.output_dir_widget.value
        os.makedirs(output_dir, exist_ok=True)

        # Seçilen model ayarları
        selected_model = self.model_widget.value
        ckpt = self.model_options[selected_model]['ckpt']
        config = self.model_options[selected_model]['config']

        # Input dizinindeki tüm wav dosyalarını işle
        for filename in os.listdir(input_dir):
            if filename.endswith('.wav'):
                # Girdi dosyasının tam yolu
                input_path = os.path.join(input_dir, filename)

                # Çıktı dosyasının adını oluştur
                output_filename = f"{os.path.splitext(filename)[0]}_upscaled.wav"
                output_path = os.path.join(output_dir, output_filename)

                # İşleme komutu
                cmd = f"python /content/Apollo/inference.py " \
                      f"--in_wav '{input_path}' " \
                      f"--out_wav '{output_path}' " \
                      f"--chunk_size {self.chunk_size_widget.value} " \
                      f"--overlap {self.overlap_widget.value} " \
                      f"--ckpt '{ckpt}' " \
                      f"--config '{config}'"

                print(f"İşleniyor: {filename}")
                os.system(cmd)

        self.display_output_sounds(output_dir)

    def display_output_sounds(self, output_dir):
        print("\n🎵 Çıktı Ses Dosyaları 🎵")
        print("-" * 50)

        # Tüm wav dosyalarını sırala ve oynat
        wav_files = [f for f in os.listdir(output_dir) if f.endswith('.wav')]

        for index, filename in enumerate(wav_files, 1):
            file_path = os.path.join(output_dir, filename)

            print(f"\n{index}. Dosya: {filename}")

            # Ses oynatıcıyı oluştur
            ipydisplay(Audio(file_path))

            # Dosya bilgilerini göster
            import soundfile as sf
            audio_info = sf.info(file_path)
            print(f"   • Süre: {audio_info.duration:.2f} saniye")
            print(f"   • Örnekleme Hızı: {audio_info.samplerate} Hz")
            print(f"   • Kanal Sayısı: {audio_info.channels}")
            print("-" * 50)

# Aracı başlat
upscaler = AudioUpscaler()
upscaler.display_interface()